# To Do

* ~~parallelize experimental pipeline to make experiment iterations go faster on jakku~~
* experiment with different clustering approaches
    * different values for K in KMeans
    * with and without standardization of the training data for clustering
        * [a la this post](https://datascience.stackexchange.com/questions/6715/is-it-necessary-to-standardize-your-data-before-clustering)
    * DBScan or some other clustering approach
    * stratify data across activity classes to match labeled data
* experiment with uncertainty sampling instead of random sampling of personal data
* investigate clusters
    * How big are the clusters? Is it just a matter of size of the cluster?
    * are the clusters fairly homogenous/heterogenous with respect to
        * activity
        * user data (is a user's data mostly within one cluster, or many?)
* investigate why this method works great for some people, but not others
* What does this mean for designing human-in-the-loop systems?

# Methods

## Training Methods
In this study, we use three different approaches to classification when some amount of labeled data from the individual end user is available. I'll refer to this as personal training data. The first approach is to use the personal training data exclusively regardless of how little of it is available.  The second approach is to combine personal training data with all available impersonal data (i.e. data collected from other users). Users can be very different from each other in ways that affect the relationship between the target label (e.g.: activity) and the input features. For example, we may see that users with longer legs exhibit a different motion pattern than those with shorter legs. The shorter legged person might be moving their legs forward and back at a higher frequency in order to maintain a forward speed comparable to the average walking partner while the longer legged person may be moving their legs at a lower frequency. For this reason, we introduced a new method which first uses an unsupervised clustering algorithm, such as K-Means Clustering, to partition the data into clusters in feature space. The personal training set is then compared to each cluster to determine which cluster is the closest. The closest cluster is then combined with the personal training set and used to train a new model.


## Validation Method
Our validation method is based on the one used by Lockhart et al. with some modifications to better understand how the amount of personal training data affects the model performance. All experiments in this chapter use a general framework in which one user is held out while the data from the rest of the users becomes available impersonal training data. In a live setting, the initial data from the end user will have a large impact on the model built. We do two things to control for this.  First we iterate through all the personal data with a 10-fold validation process where the data is divided into 10 folds with 1 of the folds held out as a test set for this individual.  We then randomly sample the other 9 folds for "active" labels.  These are labels that the user would have hypothetically given to the system had the system randomly asked the user for it. This sampled data is then added to the universal data to create hybrid personal-impersonal training set and a model is then trained from it. We then test this model's performance against the held-out test set from the outer k-fold process. When we are finished, the accuracy is recorded as the mean of these 5 iterations with randomly sampled active personal labels. Next we increase the size of the active sample we will take and repeat the process for the same held out test set.  When we have iterated through all of the potential amounts of training data we are considering, we continue with a different fold being held out as the test set.

For these initial experiments, the k-folds and sampling processes are stratified to balance the distribution of available classes. However, randomly selecting moments to ask the user for an activity label will not likely yield a balanced sample as we don't expect users to walk, jog, stand, sit, and use the stairs entirely equally. Subsequent experiments should use validation methods and sampling methods which attempt to account for this non-uniform performance of activities in their end-users.

# The Datasets

We are using activity recognition datasets collected by the WISDM group at Fordham University. There are two datasets, v1.1 which has all data collected from specific exercises in the lab and v2.0 in which all data was collected in the wild with data being labeled by the user in a "training mode".

# Experiment 1

In this first experiment, we use only the v1.1 dataset using the validation process describe above with each of the three methods for utilizing personal data : 1. personal data on its own, 2. personal data combine with *all available* impersonal data, and 3. personal data combined only with the *nearest cluster* of impersonal data.

In [1]:
from IPython.core.display import display, HTML
display(HTML('''<div>
    <a href="https://plot.ly/~scottofthescience/365/?share_key=op1sl4s2HmIPh2RMUQq1D7" target="_blank" title="Plot 365" style="display: block; text-align: center;"><img src="https://plot.ly/~scottofthescience/365.png?share_key=op1sl4s2HmIPh2RMUQq1D7" alt="Plot 365" style="max-width: 100%;width: 600px;"  width="600" onerror="this.onerror=null;this.src='https://plot.ly/404.png';" /></a>
    <script data-plotly="scottofthescience:365" sharekey-plotly="op1sl4s2HmIPh2RMUQq1D7" src="https://plot.ly/embed.js" async></script>
</div>'''))

In [2]:
html = '''<iframe width="900" height="800" frameborder="0" scrolling="no" src="//plot.ly/~scottofthescience/367.embed"></iframe>'''
display(HTML(html))

## Results :
The first graph above shows the performance of each approach to personalization as the amount of personal training data increases. What we see is that while each model performs about the same on average, the model using only personal training data and the model using personal training data as well as the nearest cluster in the impersonal training data have much better worst case scenarios compared to the model using all of the impersonal data.  Comparing these two against each other, the model using the impersonal cluster has much better average performance than the personal model consistently as the amount of available personal training data increases.

The second graph shows the amount of users who get the best performance from each approach as the amount of personal training data increases. This plot seeks to show that while some approaches do best on average, there are still a non-negligible amount of users who get the best performance from other approaches. Unsurprisingly, as the amount of personal data available increases, the number of users who get the best performance from a model using only personal data also increases. It is interesting, that there is even a place for the exclusively impersonal model especially with only 10 personal labels available.

# Experiment 2
The second experiment is exactly the same as the first except that we use the v2.0 dataset instead where the main difference is that the data was labeled by the individual user in their natural environment as opposed to a lab setting where the data was labeled while they were explicitly told to perform particular settings

In [3]:
html = '''<div>
    <a href="https://plot.ly/~scottofthescience/369/?share_key=uE1l8EWMfoRZAPScsTVBSW" target="_blank" title="Plot 369" style="display: block; text-align: center;"><img src="https://plot.ly/~scottofthescience/369.png?share_key=uE1l8EWMfoRZAPScsTVBSW" alt="Plot 369" style="max-width: 100%;width: 600px;"  width="600" onerror="this.onerror=null;this.src='https://plot.ly/404.png';" /></a>
    <script data-plotly="scottofthescience:369" sharekey-plotly="uE1l8EWMfoRZAPScsTVBSW" src="https://plot.ly/embed.js" async></script>
</div>
'''
display(HTML(html))

In [4]:
html = '''<div>
    <a href="https://plot.ly/~scottofthescience/371/?share_key=bXaK20xk31V4soUOOsvFWi" target="_blank" title="Plot 371" style="display: block; text-align: center;"><img src="https://plot.ly/~scottofthescience/371.png?share_key=bXaK20xk31V4soUOOsvFWi" alt="Plot 371" style="max-width: 100%;width: 600px;"  width="600" onerror="this.onerror=null;this.src='https://plot.ly/404.png';" /></a>
    <script data-plotly="scottofthescience:371" sharekey-plotly="bXaK20xk31V4soUOOsvFWi" src="https://plot.ly/embed.js" async></script>
</div>
'''
display(HTML(html))

## Results :
With the WISDM v2.0 dataset in which labels are collected by the end users themselves, we see fairly similar results to using the WISDM v1.1 which was collected in the lab, with a few differences. Here, the model using only personal data takes performs better on average than either model that uses impersonal data until the amount of personally labeled training data starts to go above 80 samples. We expected that with increasing personally labeled data comes increasing model personalization and thus increased model performance, but this appears to show that the increase in personally labeled data is increasing model performance more when combined with impersonal data in some way.  This could be an effect of the ensemble approach that random forest decision trees take which usually result in reasonable generalization given enough of the right kind of data, but a further analysis of the resulting decision boundary in particular models will be needed to be sure.

The second graph appears to show that the amount of users who get their best performance from each model remains about the same even as personally labeled training data is added to the model

# Experiment 3
The third experiment is perhaps the most realistic for initial applications of activity recognition where the initial data is collected in a lab setting and deployed with the initial version of the application. Often this is done so that initial users who have not yet provided personalized training data can benefit from the impersonal data and will experience further improvements in accuracy as they provide more labeled data

In [6]:
html = '''<div>
    <a href="https://plot.ly/~scottofthescience/373/?share_key=iScx69G40XljOqIIU6T4r5" target="_blank" title="Plot 373" style="display: block; text-align: center;"><img src="https://plot.ly/~scottofthescience/373.png?share_key=iScx69G40XljOqIIU6T4r5" alt="Plot 373" style="max-width: 100%;width: 600px;"  width="600" onerror="this.onerror=null;this.src='https://plot.ly/404.png';" /></a>
    <script data-plotly="scottofthescience:373" sharekey-plotly="iScx69G40XljOqIIU6T4r5" src="https://plot.ly/embed.js" async></script>
</div>
'''
display(HTML(html))

In [7]:
html = '''<div>
    <a href="https://plot.ly/~scottofthescience/375/?share_key=cGkgkPxUPX7e2QIPwiHUBA" target="_blank" title="Plot 375" style="display: block; text-align: center;"><img src="https://plot.ly/~scottofthescience/375.png?share_key=cGkgkPxUPX7e2QIPwiHUBA" alt="Plot 375" style="max-width: 100%;width: 600px;"  width="600" onerror="this.onerror=null;this.src='https://plot.ly/404.png';" /></a>
    <script data-plotly="scottofthescience:375" sharekey-plotly="cGkgkPxUPX7e2QIPwiHUBA" src="https://plot.ly/embed.js" async></script>
</div>
'''
display(HTML(html))



### Results with v2.0 dataset :


### Results trained with v1.1 applied to v2.0:
Impersonal + Personal model performs nearly perfectly with every amount of personal data. Why is this?

** We assume that the labels are noisier in v2.0 data for two reasons: 1. The data was collected in the wild where users are more likely to input the correct activity incorrectly. 2. Training on this dataset and testing on a held out sample from the same dataset yields a result that is not promising. *How can this model be doing so well with noisy labels? You would think that there would be some more error due simply to user mislabeling* **


### Follow-up Question #1: 
*Q: What is different about the moments when this approach works and when it doesn't work?*

### Follow-up Question #2: 
*Q1 : Can we determine early in the use of the AR technology whether the personal, universal + personal, or cluster + personal approach will work best?*
* Reason about this question along with the Tong Yu, Yong Zhuang, Ole Mengshoel, Osman Yagan paper (Hybridizing Personal and Impersonal Machine Learning Models for Activity Recognition On Mobile Devices)

### Follow-up Question #3: 
For both the experiment in which the model was trained on v2.0 data and the experiment in which the model was trained on v1.1 and test on v2.0 data, the personal model did best for a surprisingly non-negligible amount of people when the dataset is smallest. This is counter-intuitive given that there is very very little data for the model to leverage here. 

*Q1 : Why does this work so well in the second two experiments?*

*Q2 : Why didn't this work well for the first experiment?*

### Follow-up Question #4:
Q : How do the class distributions in v2.0 differ from v1.1 and could that be affecting the differences in model performance? 

# Experiment 2
Using an uncertainty sampling approach to intelligently select data from available personal training set.